<a href="https://colab.research.google.com/github/WittJonas/bundestag/blob/main/bundestag_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install transformers --quiet

In [ ]:
pip install datasets --quiet

     |████████████████████████████████| 264 kB 8.3 MB/s 
     |████████████████████████████████| 118 kB 64.2 MB/s 
     |████████████████████████████████| 243 kB 63.1 MB/s 


In [ ]:
import pandas as pd
import tensorflow as tf
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

from datasets import Dataset
from datasets import load_dataset
from datasets import load_metric
from transformers import TrainingArguments

from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import BertForSequenceClassification
from transformers import TrainingArguments
from transformers import Trainer
from transformers import EarlyStoppingCallback

In [ ]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [ ]:
# from datasets import load_dataset
# imdb = load_dataset('imdb', ignore_verifications=True)

In [ ]:
test = pd.read_csv("drive/My Drive/NLP_project/test.csv")
train = pd.read_csv("drive/My Drive/NLP_project/train.csv")

dataset = load_dataset('csv', data_files={'train': ['drive/My Drive/NLP_project/train.csv'],
                                              'test': 'drive/My Drive/NLP_project/test.csv'})

Using custom data configuration default-7a78e5707f5e4a88


0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-7a78e5707f5e4a88/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff. Subsequent calls will reuse this data.


In [ ]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-german-cased")

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/433 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/255k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/485k [00:00<?, ?B/s]

In [ ]:
# Run tokenizer 
def tokenize_function(examples):
    return tokenizer(examples["Tweets"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

train_dataset = tokenized_datasets["train"]
eval_dataset = tokenized_datasets["test"]

  0%|          | 0/8 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [ ]:
# Load model
model = AutoModelForSequenceClassification.from_pretrained("bert-base-german-cased", num_labels=6)

In [ ]:
eval_dataset

Dataset({
    features: ['Tweets', 'attention_mask', 'input_ids', 'label', 'token_type_ids'],
    num_rows: 1897
})

In [ ]:
# compute metrics
# def compute_metrics(p):    
#     pred, labels = p
#     pred = np.argmax(pred, axis=1)
#     accuracy = accuracy_score(y_true=labels, y_pred=pred, average='micro')
#     recall = recall_score(y_true=labels, y_pred=pred, average='micro')
#     precision = precision_score(y_true=labels, y_pred=pred, average='micro')
#     f1 = f1_score(y_true=labels, y_pred=pred, average='micro')
#     return {
#         "accuracy": accuracy, "precision": precision, "recall": recall, 
#             "f1": f1} 

def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  # calculate accuracy using sklearn's function
  acc = accuracy_score(labels, preds)
  return {
      'accuracy': acc,
  }

# training arguments
training_args = TrainingArguments("test_trainer", 
                                  evaluation_strategy="steps",
                                  eval_steps=500, 
                                  per_device_train_batch_size=8,
                                  per_device_eval_batch_size=8,
                                  # logging_steps=120,
                                  num_train_epochs=3,
                                  load_best_model_at_end = True)
                                  

# trainer
trainer = Trainer(
    model=model, 
    args=training_args, 
    train_dataset=train_dataset, 
    eval_dataset=eval_dataset, 
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: Tweets.
***** Running training *****
  Num examples = 7587
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 2847


Step,Training Loss,Validation Loss,Accuracy
500,0.122000,1.880334,0.761202
1000,0.713900,0.803810,0.815498
1500,0.237500,0.892833,0.834475
2000,0.193100,0.942021,0.839220
2500,0.057100,1.033089,0.837111


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: Tweets.
***** Running Evaluation *****
  Num examples = 1897
  Batch size = 8
Saving model checkpoint to test_trainer/checkpoint-500
Configuration saved in test_trainer/checkpoint-500/config.json
Model weights saved in test_trainer/checkpoint-500/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: Tweets.
***** Running Evaluation *****
  Num examples = 1897
  Batch size = 8
Saving model checkpoint to test_trainer/checkpoint-1000
Configuration saved in test_trainer/checkpoint-1000/config.json
Model weights saved in test_trainer/checkpoint-1000/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: Tweets.
***** Running Eval

TrainOutput(global_step=2500, training_loss=0.26471599044799804, metrics={'train_runtime': 2452.3539, 'train_samples_per_second': 9.281, 'train_steps_per_second': 1.161, 'total_flos': 5259778891591680.0, 'train_loss': 0.26471599044799804, 'epoch': 2.63})

## **Making Prediction**

In [ ]:
# Tokenize test data
X_test_tokenized = tokenizer(bla, padding=True, truncation=True, max_length=512) 

# Create torch dataset
test_dataset = Dataset.from_dict(X_test_tokenized) 

# Load trained model
model_path = "/content/test_trainer/checkpoint-2000"
model = BertForSequenceClassification.from_pretrained(model_path, num_labels=6)

# Define test trainer
test_trainer = Trainer(model) 

# Make prediction
raw_pred, _, _ = test_trainer.predict(test_dataset) 

# Preprocess raw predictions
y_pred = np.argmax(raw_pred, axis=1)

print(y_pred)